# **IP2_Run1**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.7 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """Your goal is to invest in several of 10 possible investment strategies in the most optimal way. The historic returns of those strategies are stored in the file "investments_data.csv". Each column represents one strategy and the rows are the past investment outcomes. There is no index and the values are separated by a ;.

The costs for investing in a given investment is stored in a vector A, which has one value for each strategy in order.
The values are: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]

You can only invest once into an investment.

Unfortunately due to other costs and inflation, your available budget at this time is uncertain. There are four possible budget scenarios with different probabilities: scenario 1 with 1000 euros and probability of 0.55, scenario 2  with 1100 euros and probability of 0.4, scenario 3 with 900 euros and probability of 0.04, scenario 4 with 1200 euros and probability of 0.01.
The tolerable probability of exceeding the budget is 0.4.

Please formulate a mean-variance mathematical model for this optimization problem, considering the past performance of investment strategies and the uncertain budget. You can take 2 as the risk parameter r."""


In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [6]:
print(response1.choices[0].message.content)

Variables:

1. Binary variable x_i: 1 if Strategy i is chosen, 0 otherwise (where i = 1 to 10).
2. Two-dimensional variable B_ij: represents the budget allocated to Strategy i in scenario j (where i = 1 to 10 and j = 1 to 4).

Auxiliary variable:

1. pf: The total performance of the investment portfolio under specific budget scenario. 
2. rp: Risk probability to achieve particular performance in specific budget scenario. 

Constants:

1. A_i: Cost of investing in Strategy i (where i = 1 to 10). The values will be [80, 340, 410, 50, 180, 221, 15, 348, 191, 225].
2. The budget scenarios are represented as B_j (where j = 1 to 4). The values specified are [1000, 1100, 900, 1200] for j = 1 to 4 respectively.
3. The probability of each budget scenario, p_j (where j = 1 to 4). The values specified are [0.55, 0.40, 0.04, 0.01] for j = 1 to 4 respectively.
4. Risk parameter, r = 2.
5. A data frame or matrix, R, is a 10*n matrix that captures the historic return of each strategy, where n is the 

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [8]:
Markdown(response2.choices[0].message.content)

Objective function:

Maximize: SUM ((p_j * SUM (R[i,:] * B_ij) / n) - r * VAR(B_ij)) over all strategies i and scenarios j

where: 

- p_j is the probability of scenario j, 
- R[i,:] is the historic outcomes of strategy i, 
- B_ij is the budget for strategy i in scenario j,
- n is the number of past outcomes,
- r is the risk parameter,
- VAR(B_ij) is the variance of the budgets for strategy i across the scenarios.

In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [10]:
Markdown(response3.choices[0].message.content)

Constraints:

1. For each strategy i: SUM (B_ij) <= A_i * x_i for all j (No strategy can receive more than its respective available budget multiplied by the decision to invest or not)

2. For each scenario j: SUM (A_i * x_i) <= B_j (The total cost of the selected investment strategies must not exceed the budget under each specific scenario)
   
3. The sum of the products of the probabilities of each scenario and the amount invested in every strategy, must not exceed the tolerable probability of exceeding the budget: SUM(p_j * SUM(A_i * x_i)) <= P_t, for all i, j

4. Binary constraint on x_i: x_i = {0,1}, for all strategies i

5. Non-negativity constraint: B_ij >= 0 for all strategies i and scenarios j.

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [12]:
Markdown(response4.choices[0].message.content)

Here is the pyomo optimization model:

```python
from pyomo.environ import *

# Creating a concrete model
model = ConcreteModel()

# Defining indexes
model.Strategies = RangeSet(1,10)
model.Scenarios = RangeSet(1,4)

# Defining parameters
model.budgets = Param(model.Strategies, initialize={1:80, 2:340, 3:410, 4:50, 5:180, 6:221, 7:15, 8:348, 9:191, 10:225}, 
                      within=NonNegativeReals)
model.scenarios = Param(model.Scenarios, initialize={1:1000, 2:1100, 3:900, 4:1200}, 
                        within=NonNegativeReals)
model.probabilities = Param(model.Scenarios, initialize={1:0.55, 2:0.4, 3:0.04, 4:0.01}, 
                            within=NonNegativeReals)

# Sample data (Replace this section with actual data)
model.returns = Param(model.Strategies, initialize= {i: (1 + ((i-1)%2)/2) for i in model.Strategies})

# Defining variables
model.strategy_selected = Var(model.Strategies, within=Binary)
model.budget_alloc = Var(model.Strategies, model.Scenarios, within=NonNegativeReals)

# Defining Objective function (Replace 4 with the number of past outcomes and 2 with the risk parameter)
model.profit = Objective(expr = sum(model.probabilities[j] * sum(model.returns[i] * model.budget_alloc[i,j] / 4 - 
                                     2 * model.budget_alloc[i,j]**2 for i in model.Strategies) 
                          for j in model.Scenarios), sense=maximize)

# Defining constraints
model.budget_alloc_const = ConstraintList()
for i in model.Strategies:
    model.budget_alloc_const.add(
        sum(model.budget_alloc[i,j] for j in model.Scenarios) <= model.budgets[i] * model.strategy_selected[i])

model.budget_total_const = ConstraintList()
for j in model.Scenarios:
    model.budget_total_const.add(
        sum(model.budgets[i] * model.strategy_selected[i] for i in model.Strategies) <= model.scenarios[j])
    
model.risk_prob_const = Constraint(expr = sum(model.probabilities[j] * sum(model.budgets[i] * model.strategy_selected[i] 
                           for i in model.Strategies) for j in model.Scenarios) <= 0.4)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the result
for i in model.Strategies:
    print("Strategy", i, "selected:", model.strategy_selected[i].value)
```

This script first builds and solves a concrete model of the posed optimization problem. It maximizes the objective function while subject to the given constraints. Note that the parameters need to correspond to the actual problem at hand. The above solver used is glpk, replace it with any other linear programming solver if required. Remember to replace the sample data with the actual data for model.returns.

### **Run the code Generated by GPT4**

In [13]:
from pyomo.environ import *

# Creating a concrete model
model = ConcreteModel()

# Defining indexes
model.Strategies = RangeSet(1,10)
model.Scenarios = RangeSet(1,4)

# Defining parameters
model.budgets = Param(model.Strategies, initialize={1:80, 2:340, 3:410, 4:50, 5:180, 6:221, 7:15, 8:348, 9:191, 10:225},
                      within=NonNegativeReals)
model.scenarios = Param(model.Scenarios, initialize={1:1000, 2:1100, 3:900, 4:1200},
                        within=NonNegativeReals)
model.probabilities = Param(model.Scenarios, initialize={1:0.55, 2:0.4, 3:0.04, 4:0.01},
                            within=NonNegativeReals)

# Sample data (Replace this section with actual data)
model.returns = Param(model.Strategies, initialize= {i: (1 + ((i-1)%2)/2) for i in model.Strategies})

# Defining variables
model.strategy_selected = Var(model.Strategies, within=Binary)
model.budget_alloc = Var(model.Strategies, model.Scenarios, within=NonNegativeReals)

# Defining Objective function (Replace 4 with the number of past outcomes and 2 with the risk parameter)
model.profit = Objective(expr = sum(model.probabilities[j] * sum(model.returns[i] * model.budget_alloc[i,j] / 4 -
                                     2 * model.budget_alloc[i,j]**2 for i in model.Strategies)
                          for j in model.Scenarios), sense=maximize)

# Defining constraints
model.budget_alloc_const = ConstraintList()
for i in model.Strategies:
    model.budget_alloc_const.add(
        sum(model.budget_alloc[i,j] for j in model.Scenarios) <= model.budgets[i] * model.strategy_selected[i])

model.budget_total_const = ConstraintList()
for j in model.Scenarios:
    model.budget_total_const.add(
        sum(model.budgets[i] * model.strategy_selected[i] for i in model.Strategies) <= model.scenarios[j])

model.risk_prob_const = Constraint(expr = sum(model.probabilities[j] * sum(model.budgets[i] * model.strategy_selected[i]
                           for i in model.Strategies) for j in model.Scenarios) <= 0.4)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the result
for i in model.Strategies:
    print("Strategy", i, "selected:", model.strategy_selected[i].value)

ValueError: Model objective (profit) contains nonlinear terms that cannot be written to LP format

In [27]:
from pyomo.environ import *
import pandas as pd
# Creating a concrete model
model = ConcreteModel()

# Defining indexes
model.Strategies = RangeSet(1,10)
model.Scenarios = RangeSet(1,4)


# Read the CSV file
df = pd.read_csv("investments_data.csv",sep=";", header=None)
headers = []
for i in range(len(df.columns)):
    headers.append(str((i+1)))
df.columns = headers
returns = dict(df)

# Defining parameters
model.budgets = Param(model.Strategies, initialize={1:80, 2:340, 3:410, 4:50, 5:180, 6:221, 7:15, 8:348, 9:191, 10:225},
                      within=NonNegativeReals)
model.scenarios = Param(model.Scenarios, initialize={1:1000, 2:1100, 3:900, 4:1200},
                        within=NonNegativeReals)
model.probabilities = Param(model.Scenarios, initialize={1:0.55, 2:0.4, 3:0.04, 4:0.01},
                            within=NonNegativeReals)
model.returns = Param(model.Strategies, initialize= returns)

# Defining variables
model.strategy_selected = Var(model.Strategies, within=Binary)
model.budget_alloc = Var(model.Strategies, model.Scenarios, within=NonNegativeReals)

# Defining Objective function (Replace 4 with the number of past outcomes and 2 with the risk parameter)
model.profit = Objective(expr = sum(model.probabilities[j] * sum(model.returns[i] * model.budget_alloc[i,j] / 4 -
                                     2 * model.budget_alloc[i,j]**2 for i in model.Strategies)
                          for j in model.Scenarios), sense=maximize)

# Defining constraints
model.budget_alloc_const = ConstraintList()
for i in model.Strategies:
    model.budget_alloc_const.add(
        sum(model.budget_alloc[i,j] for j in model.Scenarios) <= model.budgets[i] * model.strategy_selected[i])

model.budget_total_const = ConstraintList()
for j in model.Scenarios:
    model.budget_total_const.add(
        sum(model.budgets[i] * model.strategy_selected[i] for i in model.Strategies) <= model.scenarios[j])

model.risk_prob_const = Constraint(expr = sum(model.probabilities[j] * sum(model.budgets[i] * model.strategy_selected[i]
                           for i in model.Strategies) for j in model.Scenarios) <= 0.4)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print the result
for i in model.Strategies:
    print("Strategy", i, "selected:", model.strategy_selected[i].value)

ERROR:pyomo.core:Rule failed for Param 'returns' with index 1:
KeyError: "Index '1' is not valid for indexed component 'returns'"
ERROR:pyomo.core:Constructing component 'returns' from data=None failed:
    KeyError: "Index '1' is not valid for indexed component 'returns'"


KeyError: "Index '1' is not valid for indexed component 'returns'"

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**